In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("../credentials.env")

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')
SEED=123

# temperature

Defaults to 1

`temperature` 매개변수의 기본값은 일반적으로 1입니다. `temperature` 매개변수는 생성된 텍스트의 예측 가능성과 다양성을 조정하는 데 사용됩니다. 값이 높을수록 생성된 텍스트가 더 다양하고 예측 불가능해지며, 값이 낮을수록 보다 예측 가능하고 일관된 결과를 생성합니다.  

- 1에 가깝게 설정: 이는 기본 설정으로, 모델이 높은 다양성과 창의성을 가진 응답을 생성하도록 합니다. 이 설정은 가장 균형 잡힌 결과를 제공하며, 예측하기 어려운 다양한 응답을 생성할 수 있습니다.
- 0에 가깝게 설정: 모델은 가장 확률이 높은 단어를 선택하려고 하여, 매우 일관되고 예측 가능한 텍스트를 생성합니다.
- 1보다 크게 설정: 모델은 더 창의적이고 예측하기 어려운 텍스트를 생성하지만, 때로는 주제에서 벗어나거나 덜 일관된 결과를 낼 수도 있습니다.  

일반적으로 Temperature 또는 Top_p 값을 바꾸는걸 추천하지만, 둘 다 동시에 바꾸는 것을 추천하지 않습니다.

In [ ]:
def call_openai(num_times, messages, temperature=1, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=deployment_name,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": messages}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=deployment_name,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": messages}],
                    max_tokens=60,
                    temperature = temperature
            )
        
        print(str(i+1) + ": " + response.choices[0].message.content)

In [ ]:
call_openai(10, '이야기를 시작해줘')

In [ ]:
call_openai(10, '이야기를 시작해줘', temperature = 0)

`seed` 파라미터는 모델이 생성하는 결과의 재현성을 위해 사용됩니다. `seed` 는 난수 생성기에 사용되는 초기값이며, 동일한 `seed` 값으로 요청을 하면 모델은 동일한 입력에 대해 동일한 출력을 생성합니다. 이는 실험의 재현성을 보장하거나, 동일한 응답을 여러 사용자에게 제공할 필요가 있을 때 유용할 수 있습니다. 다음과 같은 상황에서 사용 가능합니다.  
- 재현성이 중요할 때
- 사용자 경험의 일관성
- A/B 테스팅  

재현 가능한 결과를 위해 `seed` 를 사용하는 경우, 다른 모델 파라미터(예: temperature, max_tokens 등)도 동일하게 유지해야 동일한 결과를 얻을 수 있습니다.


In [ ]:
call_openai(10, '이야기를 시작해줘', temperature = 0.7, use_seed=True)

In [ ]:
SEED=1
call_openai(10, '이야기를 시작해줘', temperature = 0.7, use_seed=True)

# top_p

Defaults to 1

`top_p` 파라미터는 생성된 텍스트의 다양성과 창의성을 조절하는 데 사용됩니다. 이는 "nucleus sampling"이라고도 알려진 방법으로, 생성 과정에서 가능성이 높은 다음 단어의 확률 분포에서 특정 누적 확률(top_p)에 도달할 때까지 가장 가능성이 높은 단어들만 고려합니다. 예를 들어, top_p를 0.9로 설정하면, 모델은 가능한 다음 단어들의 확률을 높은 것부터 낮은 것까지 정렬하고, 누적 확률이 90%에 도달할 때까지의 단어들만 고려하여 다음 단어를 선택합니다.

In [ ]:
def call_openai(num_times, messages, top_p):
    for i in range(num_times):
        
        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=60,
            top_p = top_p
        )

        print(str(i+1) + ": " + response.choices[0].message.content.strip(" \n"))

In [ ]:
messages=[
    {"role": "user", "content": '이야기를 시작해줘'},
]

call_openai(10, messages, top_p = 1)

In [ ]:
call_openai(10, messages, top_p = 0.1)

# n

Defaults to 1

요청한 Prompt에 대해 Completion 몇 개를 생성할지 정하는 옵션임.

참고 :이 매개 변수는 많은 Completion을 생성하기 때문에 토큰 할당량을 빠르게 소비 할 수 있습니다. 신중하게 사용하고 max_tokens에 대한 합리적인 설정이 있는지 확인하고 중지하십시오.

In [ ]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=60,
    n=2
)

i=0
for resp in response.choices:
    print(resp.message.content.strip(" \n"))

# presence_penalty

Defaults to 0

-2.0에서 2.0 사이의 숫자입니다. 양수 값은 지금까지 텍스트에 나타나는지 여부에 따라 새 토큰에 페널티를 주어 모델이 새 주제에 대해 이야기할 가능성을 높입니다.

# frequency_penalty

Defaults to 0

-2.0에서 2.0 사이의 숫자입니다. 양수 값은 지금까지 텍스트의 기존 빈도를 기반으로 새 토큰에 페널티를 주어 모델이 동일한 줄을 그대로 반복할 가능성을 줄입니다.

# function_call

array

모델이 함수 호출에 응답하는 방식을 제어합니다. '없음'은 모델이 함수를 호출하지 않고 최종 사용자에게 응답함을 의미합니다. '자동'은 모델이 최종 사용자 또는 함수 호출 중에서 선택할 수 있음을 의미합니다. {'name': 'my_function'}을 통해 특정 함수를 지정하면 모델이 해당 함수를 강제로 호출합니다. 함수가 없는 경우 'none'이 기본값입니다. 기능이 있는 경우 '자동'이 기본값입니다.